# Notebook that analyse users's day-to-day routine



In [13]:
from glob import glob
import re
import pandas as pd
from wenet_models import LocationPoint

In [2]:
locations_glob_expr = '/idiap/temp/wdroz/locations/*.csv'

In [5]:
all_location_files = glob(locations_glob_expr)
len(all_location_files)

241

In [20]:
users_windows = dict()
user_regex = re.compile(r'\/([^/\\_]+)_location\.csv')

In [53]:
for location_file in all_location_files:
    df = pd.read_csv(location_file)
    df['date'] = pd.to_datetime(df['timestamp'], unit='s')
    df = df.set_index('date')
    df = df[~df.index.duplicated(keep='first')]
    current_user = re.search(user_regex, location_file).group(1)
    print(current_user)
    break  # todo remove me

02f95f41-cc1c-42d9-ab19-ef86a2fbbf4e


In [54]:
df.head()

,userid,night,type,timestamp,timezone,local_time,source,latitude,longitude,speed,accuracy,provider,bearing
date,,,,,,,,,,,,,
2014-09-26 21:49:42,02f95f41-cc1c-42d9-ab19-ef86a2fbbf4e,20140926,Location,1411768182,7200,20140926234942,357197052974055,47.254089,8.868810,0.0,20.0,network,0.0
2014-09-26 21:50:02,02f95f41-cc1c-42d9-ab19-ef86a2fbbf4e,20140926,Location,1411768202,7200,20140926235002,357197052974055,47.254088,8.868793,0.0,20.0,network,0.0
2014-09-26 21:50:22,02f95f41-cc1c-42d9-ab19-ef86a2fbbf4e,20140926,Location,1411768222,7200,20140926235022,357197052974055,47.254091,8.868790,0.0,20.0,network,0.0
2014-09-26 21:52:42,02f95f41-cc1c-42d9-ab19-ef86a2fbbf4e,20140926,Location,1411768362,7200,20140926235242,357197052974055,47.254071,8.868794,0.0,20.0,network,0.0
2014-09-26 21:53:02,02f95f41-cc1c-42d9-ab19-ef86a2fbbf4e,20140926,Location,1411768382,7200,20140926235302,357197052974055,47.254075,8.868798,0.0,20.0,network,0.0


In [63]:
df.groupby('night').resample('30T').median()

night     timestamp  timezone  \
night    date                                                      
20140926 2014-09-26 21:30:00  20140926.0  1.411768e+09    7200.0   
         2014-09-26 22:00:00  20140926.0  1.411769e+09    7200.0   
20140927 2014-09-27 19:00:00  20140927.0  1.411845e+09    7200.0   
         2014-09-27 19:30:00  20140927.0  1.411847e+09    7200.0   
         2014-09-27 20:00:00  20140927.0  1.411849e+09    7200.0   
         2014-09-27 20:30:00  20140927.0  1.411850e+09    7200.0   
         2014-09-27 21:00:00         NaN           NaN       NaN   
         2014-09-27 21:30:00         NaN           NaN       NaN   
         2014-09-27 22:00:00         NaN           NaN       NaN   
         2014-09-27 22:30:00  20140927.0  1.411859e+09    7200.0   

                                local_time        source   latitude  \
night    date                                                         
20140926 2014-09-26 21:30:00  2.014093e+13  3.571971e+14  47.254088   
         2014-09-26 22:00:00  2.014093e+13  3.571971e+14  47.254089   
20140927 2014-09-27 19:00:00  2.014093e+13  3.571971e+14  47.260350   
         2014-09-27 19:30:00  2.014093e+13  3.571971e+14  47.225848   
         2014-09-27 20:00:00  2.014093e+13  3.571971e+14  47.225373   
         2014-09-27 20:30:00  2.014093e+13  3.571971e+14  47.225367   
         2014-09-27 21:00:00           NaN           NaN        NaN   
         2014-09-27 21:30:00           NaN           NaN        NaN   
         2014-09-27 22:00:00           NaN           NaN        NaN   
         2014-09-27 22:30:00  2.014093e+13  3.571971e+14  47.253960   

                              longitude  speed  accuracy     bearing  
night    date                                                         
20140926 2014-09-26 21:30:00   8.868794   0.00      20.0    0.000000  
         2014-09-26 22:00:00   8.868799   0.00      20.0    0.000000  
20140927 2014-09-27 19:00:00   8.852665   0.50      24.0   75.800003  
         2014-09-27 19:30:00   8.817855   0.75      28.0  144.100006  
         2014-09-27 20:00:00   8.816466   0.00      20.0    0.000000  
         2014-09-27 20:30:00   8.816485   0.00      27.0    0.000000  
         2014-09-27 21:00:00        NaN    NaN       NaN         NaN  
         2014-09-27 21:30:00        NaN    NaN       NaN         NaN  
         2014-09-27 22:00:00        NaN    NaN       NaN         NaN  
         2014-09-27 22:30:00   8.868631   0.00      20.0    0.000000